<a href="https://colab.research.google.com/github/icecat14159/PL-Repo./blob/main/%E8%BF%BD%E7%95%AA%E6%B8%85%E5%96%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread #GoogleSheet

In [ ]:
!pip install gradio

In [ ]:
!pip install pandas #資料處理

In [1]:
from google.colab import auth #身分驗證
from google.auth import default #憑證
import gspread
import gradio as gr
import pandas as pd
import datetime #日期
import io #io

In [62]:
# 授權連線 Google Sheet
auth.authenticate_user()  #要求授權
creds, _ = default()  #獲取憑證
gc = gspread.authorize(creds) #授權

In [64]:
# 連結試算表
sheet_url = "https://docs.google.com/spreadsheets/d/1fGjPVqPHt3flo-LxBNU9EZl_ZMvg-I8UXfMhIY-jDNA/edit?usp=sharing"  #GoogleSheet連結
sh = gc.open_by_url(sheet_url)  #進入試算表
worksheet = worksheet = sh.worksheet("小說清單")

In [81]:
# 讀取試算表
COLUMNS = ["ID", "作品名稱", "作者", "評級", "進度", "狀態", "最後紀錄日期"]  #如果工作表為空時回傳
def read_data():
  records = worksheet.get_all_records()
  df = pd.DataFrame(records)
  if df.empty:
    df = pd.DataFrame(columns=COLUMNS)
  if "ID" in df.columns: #確保ID
    df["ID"] = pd.to_numeric(df["ID"], errors="coerce")
  return df
# read_data()

In [79]:
# 寫回試算表
def write_data(df):
  worksheet.clear()
  worksheet.append_row(COLUMNS)
  worksheet.append_rows(df.values.tolist())
# write_data(read_data())

In [80]:
# 新增紀錄
def add_record(name, author, rating, progress, condition, date):
  df = read_data()
  if not date:
      date = datetime.date.today().strftime("%Y/%m/%d")
  if df.empty or df["ID"].dropna().empty: #產生ID
    new_id = 1
  else:
    new_id = int(df["ID"].max()) + 1
  new_entry = {"ID": new_id, "作品名稱": name, "作者": author, "評級": rating, "進度": progress, "狀態": condition, "最後紀錄日期": date}
  df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)
  write_data(df)
  return df

# add_record(name="測試作品A", author="測試作者", rating="B", progress="第1集", condition="未完結", date="")

In [85]:
# 修改紀錄
def edit_record(record_id, name, author, rating, progress, condition, date):
  df = read_data()
  if record_id not in df["ID"].values:
    return "找不到此 ID"
  if not date:
    date = datetime.date.today().strftime("%Y/%m/%d")

  df.loc[df["ID"] == record_id, ["作品名稱", "作者", "評級", "進度", "狀態", "最後紀錄日期"]] = \
      [name, author, rating, progress, condition, date]
  write_data(df)
  return df
# edit_record(1, "作品A-修正", "作者A", "A", "30", "未完結", "")

In [86]:
# 刪除紀錄
def delete_record(record_id):
  df = read_data()
  if record_id not in df["ID"].values:
    return "找不到此 ID"
  df = df[df["ID"] != record_id]
  write_data(df)
  return df
delete_record(2)

,ID,作品名稱,作者,評級,進度,狀態,最後紀錄日期
0,1,作品A-修正,作者A,A,30,未完結,2025/12/16
2,3,測試作品A,測試作者,B,第1集,未完結,2025/12/16
